# Proyecto: Modelos No Lineales para Pronósticos MPST

## 1. Configuración inicial

En esta sección cargamos las librerías y ajustamos el entorno para el análisis, la visualización y el modelado de series de tiempo.



In [ ]:
# 1. Configuración inicial: imports y entorno

import os
import datetime
import warnings
import gc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score,
)

from sklearn.svm import SVC
from sklearn.ensemble import (
    RandomForestClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import (
    Input, Dense, LSTM, GRU, Conv1D, MaxPooling1D, Flatten,
    Dropout, BatchNormalization, TimeDistributed, ConvLSTM2D, Reshape
)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import optuna
import joblib

# Ajustes de visualización
plt.rcParams['figure.figsize'] = (16, 6)
plt.rcParams['figure.dpi'] = 100
warnings.filterwarnings("ignore")

# Semilla para reproducibilidad
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Gestión de memoria y GPU
gc.enable()
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
